# Problème torsion d'arbres 

## Librairies

In [1]:
# Importation des librairies
import dolfinx  # Module principal de FEniCSx pour le calcul par éléments finis
from dolfinx import mesh, fem, plot, io, default_scalar_type  # Sous-modules FEniCSx essentiels
from dolfinx.fem.petsc import LinearProblem  # Résolution de systèmes linéaires
from mpi4py import MPI  # Interface pour le calcul parallèle
import ufl  # Unified Form Language pour les formulations variationnelles
import numpy as np  # Calcul numérique efficace
import matplotlib.pyplot as plt
import math  # Module de fonctions mathématiques standard de Python
import pyvista as pv  # Visualisation 3D scientifique
import gmsh  # Générateur de maillage 3D
import meshio  # Lecture/écriture de différents formats de maillage
from dolfinx.io import XDMFFile  # Gestion des fichiers XDMF pour données volumineuses
import dolfinx.fem as fem  # Module FEniCSx pour la méthode des éléments finis
import dolfinx.mesh as mesh  # Module FEniCSx pour la gestion avancée des maillages
from petsc4py import PETSc  # Suite de solveurs numériques pour problèmes à grande échelle
import panel as pn
from myst_nb import glue
# Utiliser Panel pour créer un rendu interactif
pn.extension('vtk')

In [2]:
import gmsh
import numpy as np
import meshio
from dolfinx.io import XDMFFile
from mpi4py import MPI
import ipywidgets as widgets
from IPython.display import display
import pyvista as pv
from dolfinx import plot

# Initialisation du plotteur PyVista en dehors de la fonction
p = pv.Plotter(notebook=True)
p.set_background("grey")

# Fonction pour créer et visualiser la géométrie
def create_ellipse(a, b, h=1.0, lc=0.02):
    gmsh.initialize()
    gmsh.option.setNumber("General.Terminal", 0)
    gmsh.model.add("ellipse")

    # Création de l'ellipse
    ellipse = gmsh.model.occ.addEllipse(0, 0, 0, a, b)
    curve_loop = gmsh.model.occ.addCurveLoop([ellipse])
    surface = gmsh.model.occ.addPlaneSurface([curve_loop])

    # Extrusion pour obtenir un cylindre elliptique
    gmsh.model.occ.extrude([(2, surface)], 0, 0, h)
    gmsh.model.occ.synchronize()

    # Configuration du maillage
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)
    gmsh.model.mesh.generate(3)

    # Sauvegarde et conversion du maillage
    gmsh.write("ellipse.msh")
    msh = meshio.read("ellipse.msh")
    meshio.write("ellipse.xdmf", meshio.Mesh(points=msh.points, cells={"tetra": msh.cells_dict.get("tetra", [])}))
    gmsh.finalize()

    # Lecture du maillage avec FEniCSx
    with XDMFFile(MPI.COMM_WORLD, "ellipse.xdmf", "r") as xdmf_file:
        domain = xdmf_file.read_mesh(name="Grid")
        domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)
    
    # Conversion du maillage pour la visualisation avec PyVista
    u_topology, u_cell_types, u_geometry = plot.vtk_mesh(domain)
    u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

    # Effacer la scène précédente et ajouter le nouveau maillage
    p.clear()
    # Ajout du maillage à la scène de visualisation
    p.add_mesh(u_grid, 
               show_edges=True,  # Affiche les arêtes du maillage
               scalar_bar_args={
                   "title": "u",  # Titre de la barre de couleur
                   "title_font_size": 24,
                   "label_font_size": 22,
                   "shadow": True,
                   "italic": True,
                   "font_family": "arial",
                   "vertical": False  # Orientation horizontale de la barre de couleur
               })

    # Ajout d'un titre à la visualisation
    p.add_text("Cylindre Mesh", font_size=24, color="black", position="upper_edge")

    # Ajout des limites de la boîte englobante
    p.show_bounds(color="black")

    # Ajout des axes de coordonnées
    p.add_axes(color="black")

    # Définition de la couleur de fond
    p.set_background("grey")
    
    # Mise à jour de la scène
    p.show()

# Champs de saisie pour a, b, et h
a_text = widgets.FloatText(value=0.2, description='Demi-grand axe (a)')
b_text = widgets.FloatText(value=0.2, description='Demi-petit axe (b)')
h_text = widgets.FloatText(value=1.0, description='Hauteur (h)')

# Interface utilisateur et fonction interactive
ui = widgets.VBox([a_text, b_text, h_text])
out = widgets.interactive_output(create_ellipse, {'a': a_text, 'b': b_text, 'h': h_text})

display(ui, out)


Output()

## Géométrie

### Définition de la géométrie et du maillage

### Visualisation de la géométrie

In [ ]:
# Lecture du fichier .msh avec meshio
msh = meshio.read("ellipse.msh")

# Conversion et sauvegarde du maillage au format XDMF (compatible avec FEniCSx)
meshio.write("ellipse.xdmf", meshio.Mesh(points=msh.points, cells={"tetra": msh.cells_dict.get("tetra", [])}))

# Lecture du maillage XDMF avec FEniCSx
with XDMFFile(MPI.COMM_WORLD, "ellipse.xdmf", "r") as xdmf_file:
    domain = xdmf_file.read_mesh(name="Grid")
    domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)

# Affichage des informations sur le maillage
print("Maillage créé et importé avec succès.")
print(f"Nombre de sommets : {domain.topology.index_map(domain.topology.dim).size_global}")
print(f"Nombre d'éléments : {domain.topology.index_map(domain.topology.dim-1).size_global}")

In [ ]:
# Préparation du maillage pour la visualisation avec PyVista
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(domain)

# Création d'une grille non structurée PyVista à partir des données du maillage FEniCSx
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

# Initialisation de l'environnement de visualisation PyVista
p = pv.Plotter()

# Ajout du maillage à la scène de visualisation
p.add_mesh(u_grid, 
           show_edges=True,  # Affiche les arêtes du maillage
           scalar_bar_args={
               "title": "u",  # Titre de la barre de couleur
               "title_font_size": 24,
               "label_font_size": 22,
               "shadow": True,
               "italic": True,
               "font_family": "arial",
               "vertical": False  # Orientation horizontale de la barre de couleur
           })

# Ajout d'un titre à la visualisation
p.add_text("Cylindre Mesh", font_size=24, color="black", position="upper_edge")

# Ajout des limites de la boîte englobante
p.show_bounds(color="black")

# Ajout des axes de coordonnées
p.add_axes(color="black")

# Définition de la couleur de fond
p.set_background("grey")

# Affichage de la visualisation
p.show()

# Changement de la vue pour une perspective Y-Z
p.view_yz()

In [ ]:
interactive_panel = pn.pane.VTK(p.ren_win, width=755, height=400)
from myst_nb import glue
glue("img", interactive_panel)

{glue:}`img`

In [ ]:
interactive_panel

## Configuration du problème

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Exemple de slider pour changer le rayon de la poutre
rayon = widgets.FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description="Rayon")
display(rayon)

# Code Fenics ou autre qui dépend de "rayon.value"


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Exemple de slider pour changer le rayon de la poutre
rayon = widgets.FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description="Rayon")
display(rayon)

# Code Fenics ou autre qui dépend de "rayon.value"


### Définition de l'espace fonctionnel 

In [ ]:
# Exemple de dropdown pour choisir un matériau
materiau = widgets.Dropdown(
    options=['Acier', 'Aluminium', 'Plastique'],
    value='Acier',
    description='Matériau:',
)

# Fonction pour mettre à jour le slider en fonction du matériau choisi
def update_slider(change):
    if change['new'] == 'Acier':
        rayon.value = 0.05  # Valeur par défaut pour l'acier
    elif change['new'] == 'Aluminium':
        rayon.value = 0.04  # Valeur par défaut pour l'aluminium
    else:
        rayon.value = 0.03  # Valeur par défaut pour le plastique

materiau.observe(update_slider, names='value')
imk=materiau.observe(update_slider, names='value')

display(materiau)


In [ ]:
# Utiliser Panel pour créer un rendu interactif
pn.extension('vtk')
interactive_panel = pn.pane.VTK(imk.ren_win, width=700, height=400)
from myst_nb import glue
glue("img", interactive_panel)

### Définition des frontière du domaine

In [ ]:
# Définition des fonctions pour identifier les différentes faces du cylindre
def down_face(x):
    """Identifie la face inférieure du cylindre (z = 0)"""
    return np.isclose(x[2], 0)

def top_face(x):
    """Identifie la face supérieure du cylindre (z = h)"""
    return np.isclose(x[2], h)

def lateral_face(x):
    """
    Identifie la surface latérale du cylindre elliptique
    Utilise l'équation de l'ellipse : x^2/a^2 + y^2/b^2 = 1
    """
    tolerance = 1e-5  # Tolérance pour la comparaison numérique
    return (np.isclose((x[0]**2 / a**2 + x[1]**2 / b**2), 1.0, atol=tolerance)) & (0 <= x[2]) & (x[2] <= h)

# Dimension des facettes (2D pour un domaine 3D)
fdim = domain.topology.dim - 1

# Localisation des entités (facettes) correspondant à chaque face
Sigma_l = mesh.locate_entities(domain, fdim, lateral_face)
Sigma_0 = mesh.locate_entities(domain, fdim, down_face)
Sigma_h = mesh.locate_entities(domain, fdim, top_face)

# Préparation des marqueurs de facettes pour les conditions aux limites
marked_facets = np.hstack([Sigma_h])  # Utilisation uniquement de la face supérieure
marked_values = np.hstack([np.full_like(Sigma_h, 1)])  # Marqueur 1 pour la face supérieure

# Tri et création des tags de maillage
sorted_facets = np.argsort(marked_facets)
facet_tag = dolfinx.mesh.meshtags(domain, domain.topology.dim - 1, 
                                  marked_facets[sorted_facets], 
                                  marked_values[sorted_facets])

# Définition de la mesure pour les facettes marquées
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag)

### Visualisation des frontière du domaine

In [ ]:
# Définition d'une fonction pour appliquer des marqueurs aux facettes
def apply_marker(boundary_facets, marker_array, marker_value):
    """
    Applique un marqueur spécifique aux facettes de la frontière.
    
    Args:
    boundary_facets: Indices des facettes de la frontière
    marker_array: Tableau des marqueurs à mettre à jour
    marker_value: Valeur du marqueur à appliquer
    """
    # Filtrer les facettes pour ne garder que celles appartenant au domaine local
    boundary_facets = boundary_facets[boundary_facets < num_cells_local]
    # Appliquer le marqueur
    marker_array[boundary_facets] = marker_value

# Obtenir le nombre de cellules locales (important pour le calcul parallèle)
num_cells_local = domain.topology.index_map(fdim).size_local

# Initialiser les tableaux de marqueurs pour chaque type de frontière
# On crée 4 tableaux : 3 pour chaque face et 1 pour la combinaison
markers = [np.zeros(num_cells_local, dtype=np.int32) for _ in range(4)]

# Appliquer les marqueurs spécifiques à chaque face
apply_marker(Sigma_0, markers[0], 1)  # Face inférieure (z = 0)
apply_marker(Sigma_h, markers[1], 2)  # Face supérieure (z = h)
apply_marker(Sigma_l, markers[2], 3)  # Surface latérale

# Calculer le marqueur combiné
markers[3] = markers[0] + markers[1] + markers[2]

# Ajouter un marqueur spécifique (4) pour les cellules non marquées
# Cela peut être utile pour identifier le volume intérieur, par exemple
markers[3][markers[3] == 0] = 4

# Créer la connectivité topologique pour les facettes
# Ceci est nécessaire pour certaines opérations sur le maillage
domain.topology.create_connectivity(fdim, fdim)

# Obtenir les données de maillage au format compatible avec PyVista
topology, cell_types, x = dolfinx.plot.vtk_mesh(domain, fdim, np.arange(num_cells_local, dtype=np.int32))

In [ ]:
# Création du maillage PyVista à partir des données FEniCSx
grid = pv.UnstructuredGrid(topology, cell_types, x)

def add_plot(ax, marker, color, title, threshold_min):
    """
    Fonction pour ajouter des maillages à une sous-fenêtre de visualisation.
    
    Args:
    ax: Axe de la sous-fenêtre PyVista
    marker: Tableau des marqueurs pour les cellules
    color: Couleur pour les cellules filtrées
    title: Titre de la sous-fenêtre
    threshold_min: Valeur minimale pour le filtrage des cellules
    """
    # Mise à jour des données de cellule du maillage avec les marqueurs
    grid.cell_data["Marker"] = marker
    grid.set_active_scalars("Marker")
    
    # Ajout du maillage complet avec les marqueurs
    ax.add_mesh(grid, show_edges=True, color="cyan", scalar_bar_args={
        "title": "Boundary Marker",
        "title_font_size": 24,
        "label_font_size": 22,
        "shadow": True,
        "italic": True,
        "font_family": "arial",
        "vertical": False
    })
    
    # Application d'un filtre basé sur le seuil pour isoler certaines cellules
    grid_filter = grid.threshold(threshold_min, scalars='Marker')
    ax.add_mesh(grid_filter, color=color, show_edges=True)
    
    # Ajout du titre et des axes à la sous-fenêtre
    ax.add_text(title, font_size=12, color="black", position="upper_edge")
    ax.add_axes(color="black")

# Configuration de la visualisation PyVista avec 4 sous-fenêtres (2x2)
pl = pv.Plotter(shape=(2, 2))

# Sous-fenêtre 1 : Affichage de toutes les frontières
pl.subplot(0, 0)
add_plot(pl, markers[3], "red", "All Boundaries", threshold_min=0.5)

# Sous-fenêtre 2 : Affichage de la frontière inférieure
pl.subplot(0, 1)
add_plot(pl, markers[0], "red", "Down Boundary", threshold_min=0.5)

# Sous-fenêtre 3 : Affichage de la frontière supérieure
pl.subplot(1, 0)
add_plot(pl, markers[1], "red", "Top Boundary", threshold_min=1.5)

# Sous-fenêtre 4 : Affichage de la frontière latérale
pl.subplot(1, 1)
add_plot(pl, markers[2], "red", "Lateral Boundary", threshold_min=2.5)

# Configuration finale et affichage
pl.set_background("grey")
pl.show()

In [ ]:
# Utiliser Panel pour créer un rendu interactif
pn.extension('vtk')
interactive_panel = pn.pane.VTK(pl.ren_win, width=600, height=400)
from myst_nb import glue
glue("img", interactive_panel)

## Conditions aux limites

In [ ]:
# Définition des conditions aux limites de Dirichlet (déplacement nul)
u_D = np.array([0, 0, 0], dtype=default_scalar_type)

# Application des conditions aux limites sur chaque face
bc1 = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, Sigma_0), V)  # Face inférieure
bc2 = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, Sigma_h), V)  # Face supérieure
bc3 = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, Sigma_l), V)  # Surface latérale

## Propriétés matériau

In [ ]:
# Définition des constantes élastiques du matériau
mu = 80e9      # Module de cisaillement (G) en Pa
lambda_ = 120e9  # Premier paramètre de Lamé (λ) en Pa

## Définition des chargements

In [ ]:
pi = math.pi  # Nombre Pi

ang = 20.0  # Angle de torsion en radians

# Calcul du taux de torsion α
alpha = ang / h
print(f"α = {alpha} m^-1")

# Calcul de la rigidité en torsion C
C = mu * pi * (R ** 4) / 2 * alpha
print(f"C = {C} kg.m^2/s")

# Calcul de la constante A pour le champ de torsion
A = 2 * C / (pi * R**4)  # A est un scalaire constant positif

# Définition des coordonnées spatiales
x = ufl.SpatialCoordinate(domain)

# Définition des composantes du champ de torsion (vecteur des Contraintes de Cauchy)
T1 = -A * x[1]  # Composante x du champ de torsion
T2 = A * x[0]   # Composante y du champ de torsion
T3 = ufl.as_ufl(0.0)  # Composante z nulle, convertie en expression UFL

# Combinaison en un champ vectoriel de torsion
T = ufl.as_vector([T1, T2, T3])

## Résolution

In [ ]:
# Définition des tenseurs de déformation et de contrainte
def epsilon(u):
    """Tenseur de déformation linéarisé"""
    return ufl.sym(ufl.grad(u))

def sigma(u):
    """Tenseur des contraintes (loi de Hooke)"""
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)

# Définition des fonctions d'essai et de test
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Définition de la force volumique (nulle dans ce cas)
f = fem.Constant(domain, default_scalar_type((0, 0, 0)))

# Formulation variationnelle du problème
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx  # Forme bilinéaire
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds(1)  # Forme linéaire

# Définition et résolution du problème linéaire
problem = LinearProblem(a, L, bcs=[bc1], 
                        petsc_options={"ksp_type": "cg", "pc_type": "jacobi"})
uh = problem.solve()

### Visualisation des résultats

### Visualisation des déplacements

In [ ]:
# Création du maillage pour PyVista basé sur les coordonnées des dofs
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(domain)

# Créez la grille PyVista et ajoutez les valeurs des dofs à la grille
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

# Attach vector values to grid and warp grid by vector
u_grid["u"] = uh.x.array.reshape((u_geometry.shape[0], 3))

# Visualisation
p = pv.Plotter()
p.add_mesh(u_grid, show_edges=False, scalar_bar_args={
    "title": "u",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})
p.add_text("Déplacements", font_size=12, color="black", position="upper_edge")
p.add_axes(color="black")
p.set_background("grey")
p.show()

In [ ]:
# Utiliser Panel pour créer un rendu interactif
pn.extension('vtk')
interactive_panel = pn.pane.VTK(p.ren_win, width=600, height=400)
from myst_nb import glue
glue("img", interactive_panel)

### Visualisation des rotations principales

In [ ]:
def omega(u):
    """Calcule le tenseur de rotation (partie antisymétrique du gradient de déplacement)"""
    return 0.5 * (ufl.grad(u).T - ufl.grad(u))

# Calcul du tenseur de rotation
theta = omega(uh)

# Extraction des composantes du tenseur de rotation
theta_xy, theta_xz, theta_yz = theta[0, 1], theta[0, 2], theta[1, 2]

# Définition de l'espace fonctionnel pour les rotations (éléments discontinus d'ordre 0)
V_omega = fem.functionspace(domain, ("DG", 0))

def interpolate_rotation(rotation_component):
    """Interpole une composante de rotation sur l'espace fonctionnel"""
    expr = fem.Expression(rotation_component, V_omega.element.interpolation_points())
    rotation = fem.Function(V_omega)
    rotation.interpolate(expr)
    return rotation

# Interpolation des composantes de rotation
rotation_x = interpolate_rotation(theta_yz)
rotation_y = interpolate_rotation(-theta_xz)  # Note le signe négatif
rotation_z = interpolate_rotation(theta_xy)

# Calcul de la norme de rotation
norm_theta = ufl.sqrt(theta_xy**2 + theta_xz**2 + theta_yz**2)
rotation_norm = interpolate_rotation(norm_theta)

# Configuration de la visualisation
pl = pv.Plotter(shape=(2, 2))
dargs = dict(cmap="jet", show_scalar_bar=False)
warped = u_grid.warp_by_vector("u", factor=0)  # Pas de déformation pour visualiser les déformations

def add_subplot(row, col, data, title):
    """Ajoute un sous-plot pour une composante de rotation"""
    pl.subplot(row, col)
    warped.cell_data[title] = data.vector.array
    warped.set_active_scalars(title)
    pl.add_mesh(warped, **dargs)
    pl.add_axes(color="k")
    pl.add_text(title, color='k', font_size=12)

# Création des sous-plots
add_subplot(0, 0, rotation_norm, "Norme de rotation")
add_subplot(0, 1, rotation_x, "Rotation RX")
add_subplot(1, 0, rotation_y, "Rotation RY")
add_subplot(1, 1, rotation_z, "Rotation RZ")

# Configuration finale
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'grey'

# Ajout d'une barre de couleur globale
pl.add_scalar_bar(title="Rotation (rad)", n_labels=5, label_font_size=10, title_font_size=12, 
                  position_x=0.25, position_y=0.05, width=0.5)

# Affichage
pl.show()

### Visualisation des déplacements amplifiés

In [ ]:
# Création d'un objet Plotter PyVista pour la visualisation 3D
p = pv.Plotter()

# Application de la déformation au maillage
# Le facteur 0.04 amplifie la déformation pour une meilleure visibilité
warped = u_grid.warp_by_vector("u", factor=0.07)

# Ajout du maillage déformé à la scène
p.add_mesh(warped, 
           show_edges=False,  # Affiche les arêtes du maillage
           scalar_bar_args={
               "title": "Déplacement",  # Titre de la barre de couleur
               "title_font_size": 24,
               "label_font_size": 22,
               #"shadow": True,
               "italic": True,
               "font_family": "arial",
               "vertical": False,  # Orientation horizontale de la barre de couleur
               "n_labels": 5,  # Nombre d'étiquettes sur la barre de couleur
               #"fmt": "%.2e"  # Format scientifique pour les valeurs
           })

# Ajout d'un titre à la visualisation
p.add_text("Déplacements amplifiés", font_size=12, color="k", position="upper_edge")

# Ajout des axes de coordonnées
p.add_axes(xlabel='X', ylabel='Y', zlabel='Z', color="k")

# Configuration de l'arrière-plan et de l'éclairage
p.set_background("grey")
#p.enable_shadows()  # Ajoute des ombres pour améliorer la perception 3D

# Configuration de la caméra pour une vue optimale
p.camera_position = [(3, 3, 2), (0, 0, 0.5), (0, 0, 1)]

# Affichage de la visualisation
p.show()

### Visualisation des déplacements normalisés et selon les axes principaux 

In [ ]:
# Définition des arguments communs pour l'affichage des maillages
dargs = dict(
    scalars="u",       # Utilise le champ de déplacement 'u' pour la coloration
    cmap="jet",        # Utilise la palette de couleurs 'jet'
    show_scalar_bar=False,  # Ne pas afficher la barre de couleur pour chaque sous-plot
)

# Création d'un plotter PyVista avec une grille 2x2 de sous-plots
pl = pv.Plotter(shape=(2, 2))

# Sous-plot 0,0 : Déplacement normalisé (magnitude)
pl.subplot(0, 0)
pl.add_mesh(u_grid, **dargs)
pl.add_axes(color="black")
pl.add_text("Normalized Displacement", color='k', font_size=10)

# Sous-plot 0,1 : Déplacement selon X
pl.subplot(0, 1)
pl.add_mesh(u_grid.copy(), component=0, **dargs)
pl.add_axes(color="black")
pl.add_text("X Displacement", color='k', font_size=10)

# Sous-plot 1,0 : Déplacement selon Y
pl.subplot(1, 0)
pl.add_mesh(u_grid.copy(), component=1, **dargs)
pl.add_axes(color="black")
pl.add_text("Y Displacement", color='k', font_size=10)

# Sous-plot 1,1 : Déplacement selon Z
pl.subplot(1, 1)
pl.add_mesh(u_grid.copy(), component=2, **dargs)
pl.add_axes(color="black")
pl.add_text("Z Displacement", color='k', font_size=10)

# Configuration globale de la visualisation
pl.link_views()  # Lie les vues des sous-plots pour une navigation synchronisée
pl.camera_position = 'iso'  # Vue isométrique
pl.background_color = 'grey'  # Couleur de fond grise

# Ajout d'une barre de couleur globale
pl.add_scalar_bar(title="Displacement (m)", n_labels=5, label_font_size=10, title_font_size=12, position_x=0.25, position_y=0.05, width=0.5)

# Affichage de la visualisation
pl.show()


### Visualisation des composantes du tenseur des déformations

In [ ]:
# Calcul du tenseur de déformation
eps = epsilon(uh)

# Extraction des composantes du tenseur de déformation
eps_xx, eps_xy, eps_xz = eps[0, 0], eps[0, 1], eps[0, 2]
eps_yy, eps_yz = eps[1, 1], eps[1, 2]
eps_zz = eps[2, 2]

# Création de la grille PyVista
eps_topology, eps_cell_types, eps_geometry = dolfinx.plot.vtk_mesh(domain)
eps_grid = pv.UnstructuredGrid(eps_topology, eps_cell_types, eps_geometry)

# Définition de l'espace fonctionnel pour les déformations (éléments discontinus d'ordre 0)
V_eps = fem.functionspace(domain, ("DG", 0))

# Fonction pour interpoler les composantes de déformation
def interpolate_strain(eps_component):
    expr = fem.Expression(eps_component, V_eps.element.interpolation_points())
    strain = fem.Function(V_eps)
    strain.interpolate(expr)
    return strain

# Interpolation des composantes de déformation
strain_components = {
    "XX": interpolate_strain(eps_xx),
    "YY": interpolate_strain(eps_yy),
    "ZZ": interpolate_strain(eps_zz),
    "YZ": interpolate_strain(eps_yz),
    "XZ": interpolate_strain(eps_xz),
    "XY": interpolate_strain(eps_xy)
}

# Configuration de la visualisation
pl = pv.Plotter(shape=(2, 3))
warped = u_grid.warp_by_vector("u", factor=0)  # Pas de déformation pour visualiser les déformations

dargs = dict(cmap="jet", show_scalar_bar=False)

# Fonction pour ajouter un sous-plot
def add_subplot(row, col, component):
    pl.subplot(row, col)
    warped.cell_data[f"Epsilon_{component}"] = strain_components[component].vector.array
    warped.set_active_scalars(f"Epsilon_{component}")
    pl.add_mesh(warped, **dargs)    
    pl.add_axes(color="black")
    pl.add_text(f"Epsilon {component}", color='k', font_size=12)

# Création des sous-plots pour chaque composante
for i, component in enumerate(["XX", "YY", "ZZ", "YZ", "XZ", "XY"]):
    add_subplot(i // 3, i % 3, component)

# Configuration finale
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'grey'

# Ajout d'une barre de couleur globale
pl.add_scalar_bar(title="Strain", n_labels=5, label_font_size=10, title_font_size=12, position_x=0.25, position_y=0.05, width=0.5)

# Affichage
pl.show()

### Visualisation des composantes du tenseur des contraintes

In [ ]:
# Calculer le tenseur des contraintes
sig = sigma(uh)  # sigma est une fonction définie précédemment pour calculer les contraintes

# Extraire les composantes du tenseur des contraintes
sig_xx, sig_xy, sig_xz = sig[0, 0], sig[0, 1], sig[0, 2]
sig_yy, sig_yz = sig[1, 1], sig[1, 2]
sig_zz = sig[2, 2]

# Définir l'espace fonctionnel pour les contraintes (éléments discontinus d'ordre 0)
V_sig = fem.functionspace(domain, ("DG", 0))

# Fonction pour interpoler les composantes de contrainte
def interpolate_stress(sig_component):
    expr = fem.Expression(sig_component, V_sig.element.interpolation_points())
    stress = fem.Function(V_sig)
    stress.interpolate(expr)
    return stress

# Interpoler chaque composante de contrainte
stress_xx = interpolate_stress(sig_xx)
stress_yy = interpolate_stress(sig_yy)
stress_zz = interpolate_stress(sig_zz)
stress_yz = interpolate_stress(sig_yz)
stress_xz = interpolate_stress(sig_xz)
stress_xy = interpolate_stress(sig_xy)

# Configuration de la visualisation
pl = pv.Plotter(shape=(2, 3))  # Créer un plotter avec une grille 2x3

dargs = dict(
    cmap="jet",
    show_scalar_bar=False,
)

# Fonction pour ajouter un sous-plot
def add_subplot(row, col, stress_component, title):
    pl.subplot(row, col)
    warped.cell_data[title] = stress_component.vector.array
    warped.set_active_scalars(title)
    pl.add_mesh(warped, **dargs)    
    pl.add_axes(color="k")
    pl.add_text(title, color='k', font_size=12)

# Ajouter chaque composante de contrainte comme un sous-plot
add_subplot(0, 0, stress_xx, "Sigma XX")
add_subplot(0, 1, stress_yy, "Sigma YY")
add_subplot(0, 2, stress_zz, "Sigma ZZ")
add_subplot(1, 0, stress_yz, "Sigma YZ")
add_subplot(1, 1, stress_xz, "Sigma XZ")
add_subplot(1, 2, stress_xy, "Sigma XY")

# Configuration finale de la visualisation
pl.link_views()  # Lier les vues pour une navigation synchronisée
pl.camera_position = 'iso'  # Vue isométrique
pl.background_color = 'grey'  # Couleur de fond grise

# Ajout d'une barre de couleur globale
pl.add_scalar_bar(title="Contrainte (Pa)", n_labels=5, label_font_size=10, title_font_size=12, position_x=0.25, position_y=0.05, width=0.5)

# Afficher la visualisation
pl.show()

### Calcul des contraintes de von Mises

In [ ]:
s = sigma(uh) - 1. / 3 * ufl.tr(sigma(uh)) * ufl.Identity(len(uh))
von_Mises = ufl.sqrt(3. / 2 * ufl.inner(s, s))

V_von_mises = fem.functionspace(domain, ("DG", 0))
stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
stresses = fem.Function(V_von_mises)
stresses.interpolate(stress_expr)

### Visualisation des contraintes de von Mises 

In [ ]:
# Créez une nouvelle instance de Plotter
# Ajout des données de contrainte de von Mises au maillage déformé
warped.cell_data["VonMises"] = stresses.vector.array
# Définition de "VonMises" comme scalaire actif pour la visualisation
warped.set_active_scalars("VonMises")

# Création d'une nouvelle instance de Plotter PyVista
p = pv.Plotter()

# Ajout du maillage déformé avec la contrainte de von Mises
p.add_mesh(warped, 
           cmap="jet",  # Utilisation de la carte de couleurs "jet"
           show_edges=False,  # Ne pas afficher les arêtes du maillage
           scalar_bar_args={
               "title": "von Mises (MPa)",  # Titre de la barre de couleur
               "title_font_size": 24,  # Taille de la police du titre
               "label_font_size": 22,  # Taille de la police des étiquettes
               "shadow": True,  # Ajout d'une ombre à la barre de couleur
               "italic": True,  # Texte en italique
               "font_family": "arial",  # Police Arial
               "vertical": False  # Barre de couleur horizontale
           })

# Ajout d'un titre à la visualisation
p.add_text("Contraintes de von Mises", font_size=12, color="black", position="upper_edge")

# Ajout des axes de coordonnées en noir
p.add_axes(color="black")

# Configuration de l'arrière-plan en gris
p.set_background("grey")

# Configuration de la caméra pour une vue optimale
p.camera_position = [(3, 3, 2), (0, 0, 0.5), (0, 0, 1)]

# Affichage de la visualisation
p.show()

In [ ]:
# Utiliser Panel pour créer un rendu interactif
pn.extension('vtk')
interactive_panel = pn.pane.VTK(p.ren_win, width=600, height=400)
from myst_nb import glue
glue("img", interactive_panel)

## Analyses des résultats

### Calcul des valeurs maximales

In [ ]:
# Extraction de la valeur maximale de la rotation RZ
max_rotation_z = np.max(np.abs(rotation_z.x.array))

# Extraction de la valeur maximale de la norme de rotation
max_rotation_norm = np.max(rotation_norm.x.array)

# Conversion en degrés pour une interprétation plus intuitive
max_rotation_z_degrees = np.degrees(max_rotation_z)

# Calcul de l'angle de torsion théorique
theoretical_twist = ang * h  # ang est l'angle de torsion par unité de longueur, h est la hauteur du cylindre

# Comparaison pour RZ
relative_error_Z = abs(max_rotation_z - theoretical_twist) / theoretical_twist * 100

# Comparaison pour norme R
relative_error_N = (max_rotation_norm - theta_theoretical) / theta_theoretical * 100

In [ ]:
# Calcul du moment d'inertie polaire J pour une section circulaire
J = pi * (R ** 4) / 2

# Calcul de l'angle de torsion théorique
theta_theoretical = C * h / (mu * J)

# Affichage des résultats
print(f"Moment d'inertie polaire J : {J:.6e} m^4")
print(f"Angle de torsion théorique θ : {theta_theoretical:.6f} radians")
print(f"Angle de torsion théorique θ : {np.degrees(theta_theoretical):.2f} degrés")

# Comparaison avec la rotation maximale calculée numériquement
if 'max_rotation_z' in locals():
    relative_error = abs(max_rotation_z - theta_theoretical) / theta_theoretical * 100
    print(f"\nComparaison avec la simulation numérique en RZ:")
    print(f"Rotation Z maximale simulée : {max_rotation_z:.6f} radians")
    print(f"Rotation Z maximale simulée : {np.degrees(max_rotation_z):.2f} degrés")
    print(f"Erreur relative : {relative_error_Z:.2f}%")
    
    print(f"\nComparaison avec la simulation numérique en norme:")
    print(f"Norme de rotation maximale : {max_rotation_norm:.6f} radians")
    print(f"Norme de rotation maximale : {np.degrees(max_rotation_norm):.2f} degrés")
    print(f"Erreur relative : {relative_error_N:.2f}%")
else:
    print("\nAttention : La rotation maximale simulée n'a pas été calculée précédemment.")

# Analyse des autres composantes
max_rotation_x = np.max(np.abs(rotation_x.x.array))
max_rotation_y = np.max(np.abs(rotation_y.x.array))

print("\nValeurs maximales des composantes de rotation :")
print(f"RX max : {np.degrees(max_rotation_x):.2f} degrés")
print(f"RX max  : {max_rotation_x:.6f} radians")
print(f"RY max : {np.degrees(max_rotation_y):.2f} degrés")
print(f"RY max  : {max_rotation_y:.6f} radians")

In [ ]:
# Extraction des valeurs maximales de rotation pour chaque composante
max_rotation_x = np.max(np.abs(rotation_x.x.array))
max_rotation_y = np.max(np.abs(rotation_y.x.array))
max_rotation_z = np.max(np.abs(rotation_z.x.array))

# Affichage des résultats
print("Valeurs maximales des composantes de rotation :")
print(f"RX max : {max_rotation_x:.6f} radians ({np.degrees(max_rotation_x):.2f} degrés)")
print(f"RY max : {max_rotation_y:.6f} radians ({np.degrees(max_rotation_y):.2f} degrés)")
print(f"RZ max : {max_rotation_z:.6f} radians ({np.degrees(max_rotation_z):.2f} degrés)")

# Calcul et affichage de la norme de rotation maximale
max_rotation_norm = np.sqrt(max_rotation_x**2 + max_rotation_y**2 + max_rotation_z**2)
print(f"\nNorme de rotation maximale : {max_rotation_norm:.6f} radians ({np.degrees(max_rotation_norm):.2f} degrés)")

# Comparaison avec l'angle de torsion théorique (si disponible)
if 'theta_theoretical' in locals():
    print(f"\nAngle de torsion théorique : {theta_theoretical:.6f} radians ({np.degrees(theta_theoretical):.2f} degrés)")
    
    # Calcul de l'erreur relative
    relative_error = (max_rotation_z - theta_theoretical) / theta_theoretical * 100
    print(f"Erreur relative (RZ max vs théorie) : {relative_error:.2f}%")

# Analyse de la contribution de chaque composante
total_rotation = max_rotation_x + max_rotation_y + max_rotation_z
print("\nContribution relative de chaque composante à la rotation totale :")
print(f"RX : {max_rotation_x / total_rotation * 100:.2f}%")
print(f"RY : {max_rotation_y / total_rotation * 100:.2f}%")
print(f"RZ : {max_rotation_z / total_rotation * 100:.2f}%")


In [ ]:
import numpy as np

# Valeur analytique de la rotation maximale (ici, 20 radians)
theta_analytique = 20.0

# Valeur numérique récupérée
theta_numerique = max_rotation_z

# Calcul de l'erreur absolue
erreur_absolue = np.abs(theta_analytique - theta_numerique)

# Calcul de l'erreur relative
erreur_relative = erreur_absolue / np.abs(theta_analytique) * 100

# Affichage des résultats
print(f"Rotation analytique maximale : {theta_analytique} radians")
print(f"Rotation numérique maximale : {theta_numerique} radians")
print(f"Erreur absolue : {erreur_absolue} radians")
print(f"Erreur relative : {erreur_relative} %")


In [ ]:
 #<details> <summary>Cliquez pour voir le code d'importation</summary> 